In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string as st
import re
import string
import os
import warnings
warnings.filterwarnings("ignore")


In [142]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [122]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [97]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [98]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [99]:

def lower_and_punctuation(data):
  punctuationfree="".join([i for i in data if i not in string.punctuation])
  return punctuationfree


def find_aspect(df):
  result=[]
  for index in range(len(df)):
    #print(df['aspect'][index],df['new_text'][index])
    res = [i for i in df['text'][index] if df['aspect'][index] in i]
    result.append(res)
  #print(len(result))
  df['text'] = result
  return df


def conj_based_split(text):
  conj =['and','but','however']
  punctuation_lst =['.','!',',',':',';','?']
  conj= conj+punctuation_lst
  lst=[]
  lst2 = []
  a=1
  for word in text.split():
    if word not in conj:
      lst.append(word)
    elif (word in conj):
      sent=' '.join(lst)
      lst2.append(sent)
      lst.clear()
    if (a==len(text.split())):
      if(len(lst)):
        sent=' '.join(lst)
        lst2.append(sent)
    a=a+1
  return lst2

  
def convertList(list1):  
  str = ''
  for i in list1: 
      str += i  
  return str 

def tokenize(text):
  text = re.split('\s+' ,text)
  return [x.lower() for x in text]


def remove_small_words(text):
  return [x for x in text if len(x) > 1 ]


def remove_stopwords(text):
  output= [i for i in text if i not in stopwords]
  return output


def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text


def detokenizer(text):
  str1 = " " 
  return (str1.join(text))

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [117]:
def preprocess(df):
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['text']= df['text'].apply(lambda x: x.lower())
  df['text']= df['text'].apply(lambda x:remove_punctuation(x))
  df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))
  df['text']= df['text'].apply(lambda x:conj_based_split(x))
  df = find_aspect(df)
  df['text']= df['text'].apply(lambda x:convertList(x))
  #df.drop(df.loc[df['label']==1].index, inplace=True)

  return df


In [118]:
df = pd.read_csv('/content/train - train.csv')

In [119]:
preprocess(df)

,text,aspect,label
0,can you check whether its cancelled completely,cancelled,1
1,cannot rely on both milk delivery,milk,0
2,i get no notification,notification,0
3,calendar view were available on iphone,view,1
4,it does not let me load a clip on the scene,load,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets phonepeola ...,unable to load money using wallets,0
3997,i not able to understand the usage of all the ...,usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [123]:
df.drop(df.loc[df['label']==1].index, inplace=True)

In [124]:
x = df['text']
y = df['label']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [125]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [126]:
model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [127]:
model.score(x_test, y_test)

0.7917282127031019

In [129]:
model.predict(vec.transform(['the app fails to deliver']))

array([0])

In [130]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [131]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 120)          60000     
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 128, 120)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 176)               209088    
                                                                 
 dense_1 (Dense)             (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [132]:
y=pd.get_dummies(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [134]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
60/60 [==============================] - 49s 735ms/step - loss: 0.6337 - accuracy: 0.6357
Epoch 2/5
60/60 [==============================] - 44s 735ms/step - loss: 0.4882 - accuracy: 0.7782
Epoch 3/5
60/60 [==============================] - 44s 733ms/step - loss: 0.4236 - accuracy: 0.8110
Epoch 4/5
60/60 [==============================] - 44s 727ms/step - loss: 0.3960 - accuracy: 0.8295
Epoch 5/5
60/60 [==============================] - 44s 729ms/step - loss: 0.3601 - accuracy: 0.8405


In [135]:
model.evaluate(X_test,y_test)

26/26 [==============================] - 2s 62ms/step - loss: 0.5055 - accuracy: 0.7820


[0.5055404901504517, 0.7820196747779846]

In [136]:
df['token_text']=df['text'].apply(lambda x:tokenize(x))

In [147]:
train, test = train_test_split(df,test_size=0.2, random_state=1)
X_train = train['text'].values
X_test = test['text'].values
y_train = train['label']
y_test = test['label']

In [137]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [148]:
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

en_stopwords = set(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

In [149]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [150]:
np.random.seed(1)
pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="linear", class_weight="balanced"))

grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm.fit(X_train, y_train)
grid_svm.score(X_test, y_test)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


0.8197829024804917

In [152]:
print(grid_svm.best_score_)

0.8326319283084083


In [ ]:
test_df = pd.read_csv('/content/test - test.csv')

In [160]:
test_df.head()

,text,aspect,label
0,improve your customer service,customer service,0
1,mobile version needs a lot of improvement esp ...,mobile version,0
2,it keeps starting from zoomed in,zoomed,0
3,hey marilyn thanks for your answer the soc2 ty...,security,0
4,delanovc zoom airtable notionhq calendly apple...,apple,2


In [ ]:
preprocess(test_df)

,text,aspect
0,improve your customer service,customer service
1,mobile version needs a lot of improvement esp ...,mobile version
2,it keeps starting from zoomed in,zoomed
3,hey marilyn thanks for your answer the soc2 ty...,security
4,delanovc zoom airtable notionhq calendly apple...,apple
...,...,...
995,in a database where i have multiple views enab...,views
996,the mere act of opening the notionhq tab makes...,opening
997,i have shared it to web now,web
998,why the members who regularly buy milk has to ...,service charge


In [161]:
preds = []
for i in range(len(test_df)):
  #print(test_df['text'][i])
  pred =int( grid_svm.predict([test_df['text'][i]]))
  preds.append(pred)

In [165]:
for i in range(len(test_df)):
  test_df['label'][i]=preds[i]

In [166]:
def predict(df):
  test_df = preprocess(df)
  preds=[]
  for i in range(len(test_df)):
  #print(test_df['text'][i])
    pred=int(grid_svm.predict([test_df['text'][i]]))
    preds.append(pred)
  test_df['label']=np.array(preds)

In [167]:
predict(test_df)

In [168]:
test_df

,text,aspect,label
0,improve your customer service,customer service,0
1,mobile version needs a lot of improvement esp ...,mobile version,0
2,it keeps starting from zoomed in,zoomed,0
3,hey marilyn thanks for your answer the soc2 ty...,security,0
4,delanovc zoom airtable notionhq calendly apple...,apple,2
...,...,...,...
995,in a database where i have multiple views enab...,views,2
996,the mere act of opening the notionhq tab makes...,opening,0
997,i have shared it to web now,web,0
998,why the members who regularly buy milk has to ...,service charge,0
